# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [209]:
# Dependencies and Setup
import geoviews as gv
import geoviews.feature as gf
import hvplot.pandas
import pandas as pd
import requests
from bokeh.models import HoverTool
import requests
from shapely.geometry import shape

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
import requests
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [210]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv(r"C:\Users\ssala\Class\World_Weather_Analysis\Vacation_Search\WeatherPy_vacation.csv")

# Display sample data
vacation_df

,City ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,6,Rikitea,PF,79.77,Clouds,-23.1203,-134.9692,Chez Bianca & Benoit
1,7,Kapaa,US,80.58,Rain,22.0752,-159.3190,Pono Kai Resort
2,9,Cabo San Lucas,MX,80.62,Clear,22.8909,-109.9124,Comfort Rooms
3,10,Atuona,PF,78.73,Clouds,-9.8000,-139.0333,Pearl Resort
4,12,San Rafael,AR,84.13,Clear,-34.6177,-68.3301,Hotel Regional
...,...,...,...,...,...,...,...,...
150,672,Boa Vista,BR,85.98,Clouds,2.8197,-60.6733,Aipana Plaza Hotel
151,678,Sibolga,ID,75.25,Clouds,1.7427,98.7792,Hotel Prima Indah
152,683,Saint-Pierre,RE,81.91,Clear,-21.3393,55.4781,Tropic Hotel
153,689,San Fernando,PH,78.53,Clouds,15.0286,120.6898,Pampanga Lodge and Restaurant


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [211]:
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "CartoLight",
    color='City',
    size='Max Temp',
    alpha=0.7,
    tools=['hover'],
    hover_cols=['Hotel Name', 'Country', 'Current Description','City ID'],
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description,City ID)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [212]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City ID"] == 651]
vacation_end = vacation_df.loc[vacation_df["City ID"] == 651]
vacation_stop1 = vacation_df.loc[vacation_df["City ID"] == 237]
vacation_stop2 = vacation_df.loc[vacation_df["City ID"] == 176]
vacation_stop3 = vacation_df.loc[vacation_df["City ID"] == 668]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [213]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end])

# Display sample data
itinerary_df

,City ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
144,651,Olinda,BR,83.93,Clouds,-8.0089,-34.8553,Fidji Motel
65,237,Pitimbu,BR,82.99,Clouds,-7.4706,-34.8086,Pousada Brisa Mar
47,176,Touros,BR,80.92,Clouds,-5.1989,-35.4608,Pousada Atlântico
148,668,Mossoro,BR,85.95,Clouds,-5.1875,-37.3442,Hotel Imperial
144,651,Olinda,BR,83.93,Clouds,-8.0089,-34.8553,Fidji Motel


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [214]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['Lat', 'Lng']].copy()


# Display sample data
waypoints_df

,Lat,Lng
144,-8.0089,-34.8553
65,-7.4706,-34.8086
47,-5.1989,-35.4608
148,-5.1875,-37.3442
144,-8.0089,-34.8553


## Use GeoViews to create map that shows the four cities in the itinerary

In [215]:
# Configure the map plot by using the itineraty_df
map_plot2 = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "CartoLight",
    color='City',
    size='Max Temp',
    alpha=0.7,
    tools=['hover'],
    hover_cols=['Hotel Name', 'Country', 'Current Description','City ID'],
    frame_width=700,
    frame_height=500
)

map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description,City ID)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [216]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [217]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '912b1fe7dc98453ca4092ed607924eac',
 'waypoints': '-8.0089,-34.8553|-7.4706,-34.8086|-5.1989,-35.4608|-5.1875,-37.3442|-8.0089,-34.8553'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [306]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-34.8553, -8.0089], 'original_index': 0},
     {'location': [-34.8086, -7.4706], 'original_index': 1},
     {'location': [-35.4608, -5.1989], 'original_index': 2},
     {'location': [-37.3442, -5.1875], 'original_index': 3},
     {'location': [-34.8553, -8.0089], 'original_index': 4}],
    'units': 'metric',
    'distance': 1225627,
    'distance_units': 'meters',
    'time': 54165.012,
    'legs': [{'distance': 87608,
      'time': 4057.853,
      'steps': [{'from_index': 0,
        'to_index': 10,
        'distance': 278,
        'time': 25.128,
        'instruction': {'text': 'Drive northwest on Avenida da Saudade.'}},
       {'from_index': 10,
        'to_index': 15,
        'distance': 120,
        'time': 11.059,
        'instruction': {'text': 'Bear left onto Rua Leúcio Marquês de Almeida.'}},
       {'from_index': 15,
        'to_index': 1192,
        'distance': 62775,
        

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [390]:

# Fetch the route's legs coordinates from the JSON reponse
legs = route_response['features'][0]['properties']['legs']
all_coordinates = route_response['features'][0]['geometry']['coordinates']


## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [391]:
from shapely.geometry import LineString
import pandas as pd

route_df = pd.DataFrame(columns=['longitude'])
for feature in route_response['features']:
    coordinates = feature['geometry']['coordinates']
    for coordinate in coordinates:
        # Each coordinate is a pair of (longitude, latitude)
        long, lat = coordinate[0], coordinate[1]
        # Append the longitude and latitude values to the dataframe
        route_df = route_df.append({'longitude': long, 'latitude': lat}, ignore_index=True)




##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [392]:

# Display sample data
route_df

,longitude,latitude
0,"[-34.855412, -8.008953]","[-34.855583, -8.008599]"
1,"[-34.808598, -7.470641]","[-34.808463, -7.470635]"
2,"[-35.460721, -5.198762]","[-35.460922, -5.198647]"
3,"[-37.344355, -5.187756]","[-37.343825, -5.188074]"


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [395]:
import geoviews as gv
import holoviews as hv
import numpy as np

route_df['longitude'] = np.array(longitude)
route_df['latitude'] = np.array(latitude)

tiles = gv.tile_sources.CartoLight()

# Define the points plot
points_plot = gv.Points(route_df, kdims=['latitude', 'longitude']).options(
    color='red', size=5)

# Combine the tiles and points plot into a single GeoViews object
route_map = tiles * points_plot

# Display the plot
route_map.opts(width=600, height=400, bgcolor='lightgray', xaxis=None, yaxis=None)

ValueError: kdims: list length must be between 2 and 2 (inclusive)